In [3]:
import requests
import time
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
import threading
from queue import Queue

In [5]:
#用来存放请求线程
g_crawl_list=[]
#用来存放解析线程
g_parse_list=[]
def create_queue():
    #先创建页码队列，使得可以多线程请求url
    page_queue=Queue()
    for i in range(1,5):
        page_queue.put(i)
     
    #创建数据队列，使得获得的数据可以多线程进行解析
    data_queue=Queue()
    return page_queue,data_queue

#我们通过这个函数来创建请求线程
def create_crawl_thread(page_queue,data_queue):
    #通过创建crawl_name来方便我们了解程序在运行哪个线程
    crawl_name=['请求线程1号','请求线程2号','请求线程3号']
    for name in crawl_name:
        #这里用一个循环创建了3个线程
        tcrawl=CrawlThread(name,page_queue,data_queue)
        g_crawl_list.append(tcrawl)
        
        


#我们通过这个函数来创建解析线程
def create_parse_thread(data_queue,fp,lock):
    #通过创建paese_name来方便我们了解程序在运行哪个线程
    parse_name=['解析线程1号','解析线程2号','解析线程3号']
    for name in parse_name:
        tparse=ParseThread(name,data_queue,fp,lock)
        g_parse_list.append(tparse)

        
#线程的Class

class CrawlThread(threading.Thread):
    def __init__(self,name,page_queue,data_queue):
        super(CrawlThread,self).__init__()
        self.name=name
        self.url='http://www.fanjian.net/jianwen-'
        self.page_queue=page_queue
        self.data_queue=data_queue
        self.headers={
                        'user_agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134',
                      }
    #每个线程都要做什么呢？run函数就是每个线程具体要做些什么
    def run(self):
        print(self.name)
        while 1:
            if self.page_queue.empty():
                break
            url=self.url+str(self.page_queue.get())
            r=requests.get(url,headers=self.headers)
            print(r.text)
            #将获得的数据放到队列当中
            self.data_queue.put(r.text)
        
        
        
class ParseThread(threading.Thread):
    def __init__(self,name,data_queue,fp,lock):
        super(ParseThread,self).__init__()
        self.name=name
        self.data_queue=data_queue
        self.fp=fp
        self.lock=lock
        
    def run():
        while 1:
            if self.data_queue.empty():
                break
        #从data_queue中获取数据并解析
        data=self.data_queue.get()
        
        #写一个解析函数
        self.parse_content(data)
    
    def parse_content(data):
        items=[]
        soup=BeautifulSoup(data,'lxml')
        a_title=soup.find_all('a',target='_blank')
        #获取图片标题
        for title in a_title:
            my_title=title.text
            item={
                '标题':my_title
            }
            items.append(item)
        #写入文件，在写入文件之前要先上锁
        self.lock.acquire()
        self.fp.write(json.dumps(items,ensure_ascii=False)+'\n')
        self.lock.release()
        
        
        
        

def main():
    
    #观察网页的url，我们发现不同页数的区别在于url上的参数，所以我们创建一个队列，让这个队列里储存页码数
    #让这些页码随机分配到不同的线程中
    #创建队列函数
    page_queue,data_queue=create_queue()
    #我们需要创建请求线程，也要创建解析线程
    
    fp=open('jian.json','a')
    #创建锁
    lock=threading.Lock()
    
    
    #先创建请求线程
    create_crawl_thread(page_queue,data_queue)
    time.sleep(3)
    #再创建解析线程
    create_parse_thread(data_queue,fp,lock)
    for tcrawl in g_crawl_list:
        tcrawl.start()
        
    for tparse in g_parse_list:
        tparse.start()
    
    
    for tcrawl in g_crawl_list:
        tcrawl.join()
    
    for tparse in g_parse_list:
        tparse.join()
    fp.close()
    
    
if __name__=='__main__':
    main()

请求线程1号
请求线程2号
请求线程3号


Exception in thread 解析线程1号:
Traceback (most recent call last):
  File "C:\Users\naive\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
TypeError: run() takes 0 positional arguments but 1 was given

Exception in thread 解析线程2号:
Traceback (most recent call last):
  File "C:\Users\naive\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
TypeError: run() takes 0 positional arguments but 1 was given

Exception in thread 解析线程3号:
Traceback (most recent call last):
  File "C:\Users\naive\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
TypeError: run() takes 0 positional arguments but 1 was given



<html>
<head><title>404 Not Found</title></head>
<body bgcolor="white">
<center><h1>404 Not Found</h1></center>
<hr><center>nginx</center>
</body>
</html>
<html>
<head><title>404 Not Found</title></head>
<body bgcolor="white">
<center><h1>404 Not Found</h1></center>
<hr><center>nginx</center>
</body>
</html>


<html>
<head><title>404 Not Found</title></head>
<body bgcolor="white">
<center><h1>404 Not Found</h1></center>
<hr><center>nginx</center>
</body>
</html>

<html>
<head><title>404 Not Found</title></head>
<body bgcolor="white">
<center><h1>404 Not Found</h1></center>
<hr><center>nginx</center>
</body>
</html>

